![Test Case](./test_case_small.png)

In [1]:
statements = '[{"type":"assignment","name":"x","sympy":"3*implicit_param_0_0","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.0254,"units_valid":true}],"params":["implicit_param_0_0"]},{"type":"assignment","name":"y","sympy":"implicit_param_1_0","implicitParams":[{"name":"implicit_param_1_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.1016,"units_valid":true}],"params":["implicit_param_1_0"]},{"type":"assignment","name":"length","sympy":"sqrt((x)**(2)+(y)**(2))","implicitParams":[],"params":["x","y"]},{"type":"query","sympy":"length","units":"inch","dimensions":[0,1,0,0,0,0,0,0,0],"units_valid":true,"implicitParams":[],"params":["length"]},{"type":"assignment","name":"velocity","sympy":"(length)/(implicit_param_4_0)","implicitParams":[{"name":"implicit_param_4_0","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":10,"units_valid":true}],"params":["length","implicit_param_4_0"]},{"type":"query","sympy":"velocity","units":"","implicitParams":[],"params":["velocity"]},{"type":"query","sympy":"(implicit_param_6_0)/(implicit_param_6_1)","units":"","implicitParams":[{"name":"implicit_param_6_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":10000,"units_valid":true},{"name":"implicit_param_6_1","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":7200,"units_valid":true}],"params":["implicit_param_6_0","implicit_param_6_1"]}]'

In [2]:
import json

import sympy

from sympy.parsing.sympy_parser import parse_expr 

from sympy.physics.units.definitions.dimension_definitions import \
                                mass, length, time, current,\
                                temperature, luminous_intensity,\
                                amount_of_substance, angle, information

from sympy.physics.units.systems.si import dimsys_SI

from sympy.utilities.iterables import topological_sort

# maps from mathjs dimensions object to sympy dimensions
dim_map = {0:mass, 1:length, 2:time, 3:current, 4:temperature, 5:luminous_intensity,
           6:amount_of_substance, 7:angle, 8:information}

inv_dim_map = {str(value.name):key for key, value in dim_map.items()}

# base units as defined by mathjs
base_units = { (0, 0, 0, 0, 0, 0, 0, 0, 0) : '',
               (1, 0, 0, 0, 0, 0, 0, 0, 0) : 'kg',
               (0, 1, 0, 0, 0, 0, 0, 0, 0) : 'm',
               (0, 0, 1, 0, 0, 0, 0, 0, 0) : 'sec',
               (0, 0, 0, 1, 0, 0, 0, 0, 0) : 'ampere',
               (0, 0, 0, 0, 1, 0, 0, 0, 0) : 'kelvin',
               (0, 0, 0, 0, 0, 1, 0, 0, 0) : 'candela',
               (0, 0, 0, 0, 0, 0, 1, 0, 0) : 'mole',
               (1, 1, -2, 0, 0, 0, 0, 0, 0) : 'N',
               (0, 2, 0, 0, 0, 0, 0, 0, 0) : 'm^2',
               (0, 3, 0, 0, 0, 0, 0, 0, 0) : 'm^3',
               (1, 2, -2, 0, 0, 0, 0, 0, 0) : 'J',
               (1, 2, -3, 0, 0, 0, 0, 0, 0) : 'W',
               (1, -1, -2, 0, 0, 0, 0, 0, 0) : 'Pa',
               (0, 0, 1, 1, 0, 0, 0, 0, 0) : 'coulomb',
               (-1, -2, 4, 2, 0, 0, 0, 0, 0) : 'farad',
               (1, 2, -3, -1, 0, 0, 0, 0, 0) : 'V',
               (1, 2, -3, -2, 0, 0, 0, 0, 0) : 'ohm',
               (1, 2, -2, -2, 0, 0, 0, 0, 0) : 'henry',
               (-1, -2, 3, 2, 0, 0, 0, 0, 0) : 'siemens',
               (1, 2, -2, -1, 0, 0, 0, 0, 0) : 'weber',
               (1, 0, -2, -1, 0, 0, 0, 0, 0) : 'tesla',
               (0, 0, -1, 0, 0, 0, 0, 0, 0) : 'Hz',
               (0, 0, 0, 0, 0, 0, 0, 1, 0) : 'rad',
               (0, 0, 0, 0, 0, 0, 0, 0, 1) : 'bits' }


# map the sympy dimensional dependences to mathjs dimensions
def get_mathjs_units(dimensional_dependencies):
    mathjs_dims = [0] * 9

    all_units_recognized = True
    for name, exp in dimensional_dependencies.items():
        dim_index = inv_dim_map.get(name)
        if dim_index is None:
            # this will hapen if the user references a parameter in an equation that has not been defined
            # will eventually want to allow the user to specify the untis for an undefined parameter
            all_units_recognized = False
            break
        mathjs_dims[dim_index] += exp

    if all_units_recognized:
        mathjs_unit_name = base_units.get(tuple(mathjs_dims))

        if mathjs_unit_name is None:
            mathjs_unit_name = ""
            latex_num = ""
            latex_den = ""
            for i, exp in enumerate(mathjs_dims):
                if exp != 0:
                    key = [0] * 9
                    key[i] = 1
                    name = base_units.get(tuple(key))
                    if mathjs_unit_name == "":
                        mathjs_unit_name = f"{name}^{exp}"
                    else:
                        mathjs_unit_name = f"{mathjs_unit_name}*{name}^{exp}"

                    if exp > 0:
                        if exp != 1:
                            new_term = f"{name}^{exp}"
                        else:
                            new_term = name
                        if latex_num == "":
                            latex_num = new_term
                        else:
                            latex_num = f"{latex_num}\\cdot{new_term}"
                    else:
                        if exp != -1:
                            new_term = f"{name}^{-exp}"
                        else:
                            new_term = name
                        if latex_den == "":
                            latex_den = new_term
                        else:
                            latex_den = f"{latex_den}\\cdot{new_term}"

            if latex_den != "":
                unit_latex = f"\\left[\\frac{{{latex_num}}}{{{latex_den}}}\\right]"
            elif latex_num != "":
                unit_latex = f"\\left[{latex_num}\\right]"
            else:
                unit_latex = ""
        else:
            if mathjs_unit_name == "":
                unit_latex = ""
            else:
                unit_latex = f"\\left[{mathjs_unit_name}\\right]"

    else:
        mathjs_unit_name = ""
        unit_latex = ""

    return mathjs_unit_name, unit_latex


def get_dims(dimensions):
    dims = sympy.Mul(1, *[dim_map[int(i)]**value for i, value in enumerate(dimensions) if value != 0.0])
    return dims

def dimensional_analysis(parameters, expression):
    # sub parameter dimensions
    parameter_subs = {param['name']:get_dims(param['dimensions']) for param in parameters}
    parameter_subs[sympy.pi] = 1
    # print(parameter_subs)
    positive_only_expression = parse_expr(str(expression).replace('-', '+'))
    final_expression = positive_only_expression.subs(parameter_subs)

    try:
        result, result_latex = get_mathjs_units(dimsys_SI.get_dimensional_dependencies(final_expression))
    except TypeError:
        result = "Dimension Error"

    return result

class ParameterError(Exception):
    pass

class DuplicateAssignment(Exception):
    pass

class ReferenceCycle(Exception):
    pass

class ParsingError(Exception):
    pass

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


In [3]:
def get_sorted_statements(statements):
    defined_params = {}
    for i, statement in enumerate(statements):
        if statement["type"] == "assignment":
            if statement["name"] in defined_params:
                raise DuplicateAssignment
            else:
                defined_params[statement["name"]] = i
            
    vertices = range(len(statements))
    edges = []
    
    for i, statement in enumerate(statements):
        for param in statement["params"]:
            ref_index = defined_params.get(param)
            if ref_index is not None:
                edges.append( (ref_index, i) )
                
    try:
        sort_order = topological_sort((vertices,edges))
    except ValueError:
        print('Reference cycle detected')
        raise ReferenceCycle
        
    sorted_statements = []
    
    for i in sort_order:
        statement = statements[i]
        statement['index'] = i # original index, needed to place results in original order
        sorted_statements.append(statement)
    
    return sorted_statements

In [4]:
def get_all_parameters(statements):
    parameters = []
    for statement in statements:
        parameters.extend(statement["implicitParams"])
        
    return parameters

In [5]:
def evaluate_statements(statements):
    
    parameters = get_all_parameters(statements)
    
    statements = get_sorted_statements(statements)
    
    for index, statement in enumerate(statements):
        try:
            statement['expression'] = parse_expr(statement['sympy'])
        except SyntaxError:
            print(f"Parsing error for equation {statement['sympy']}")
            raise ParsingError


    combined_expressions = []
    for i in range(len(statements)):
        if statements[i]['type'] == "assignment":
            combined_expressions.append(None)
            continue
        temp_statements = statements[0:i+1]
        # sub equations into each other in topological order if there are more than one
        for j, statement in enumerate(reversed(temp_statements)):
            if j == 0:
                final_expression = statement['expression']
            elif statement['type'] == "assignment":
                final_expression = final_expression.subs({statement['name'] : statement['expression']})

        combined_expressions.append(final_expression)

    # sub parameter values
    parameter_subs = {param['name']:float(param['si_value']) for param in parameters if param['si_value'] is not None}
    if len(parameter_subs) < len(parameters):
        raise ParameterError

    dims = []
    values = []
    for expression in combined_expressions:
        if expression is None:
            dims.append('')
            values.append('')
        else:
            dims.append(dimensional_analysis(parameters, expression))
            value = str(expression.subs(parameter_subs).evalf())
            values.append(value if is_number(value) else '')

    sorted_values = [None]*len(statements)
    sorted_dims = [None]*len(statements)
    
    for i, statement in enumerate(statements):
        sorted_values[statement['index']] = values[i]
        sorted_dims[statement['index']] = dims[i]

    return (sorted_values, sorted_dims)

In [6]:
def get_query_values(statements):
    error = None
    
    try:
        values, dims = evaluate_statements(json.loads(statements))
    except (DuplicateAssignment, ReferenceCycle, ParameterError, ParsingError) as e:
        error = e.__class__.__name__
        values = None
        dims = None
    
    return json.dumps({"error": error, "values": values, "dims": dims})

In [7]:
get_query_values(statements)

'{"error": null, "values": ["", "", "", "0.127000000000000", "", "0.0127000000000000", "1.38888888888889"], "dims": ["", "", "", "m", "", "m^1*sec^-1", "m^1*sec^-1"]}'

In [8]:
# x+y=
statements = '[{"type":"query","sympy":"x+y","units":"","implicitParams":[],"params":["x","y"]}]'
get_query_values(statements)

'{"error": null, "values": [""], "dims": ["Dimension Error"]}'

In [9]:
# x = 10 [mm]
# y = 2 [s]
# x/y = [hours]

statements = '[{"type":"assignment","name":"x","sympy":"implicit_param_0_0","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.01,"units_valid":true}],"params":["implicit_param_0_0"]},{"type":"assignment","name":"y","sympy":"implicit_param_1_0","implicitParams":[{"name":"implicit_param_1_0","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":2,"units_valid":true}],"params":["implicit_param_1_0"]},{"type":"query","sympy":"(x)/(y)","units":"hours","dimensions":[0,0,1,0,0,0,0,0,0],"units_valid":true,"implicitParams":[],"params":["x","y"]}]'
get_query_values(statements)

'{"error": null, "values": ["", "", "0.00500000000000000"], "dims": ["", "", "m^1*sec^-1"]}'

In [10]:
# x/y =
# x = 10 [inches]
# y = 2 [s]

statements = '[{"type":"query","sympy":"(x)/(y)","units":"","implicitParams":[],"params":["x","y"]},{"type":"assignment","name":"x","sympy":"implicit_param_1_0","implicitParams":[{"name":"implicit_param_1_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.254,"units_valid":true}],"params":["implicit_param_1_0"]},{"type":"assignment","name":"y","sympy":"implicit_param_2_0","implicitParams":[{"name":"implicit_param_2_0","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":2,"units_valid":true}],"params":["implicit_param_2_0"]}]'
get_query_values(statements)


'{"error": null, "values": ["0.127000000000000", "", ""], "dims": ["m^1*sec^-1", "", ""]}'

In [11]:
# 10[s] + 1[hour] = 
statements = '[{"type":"query","sympy":"implicit_param_0_0+implicit_param_0_1","units":"","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":10,"units_valid":true},{"name":"implicit_param_0_1","dimensions":[0,0,1,0,0,0,0,0,0],"si_value":3600,"units_valid":true}],"params":["implicit_param_0_0","implicit_param_0_1"]}]'
get_query_values(statements)

'{"error": null, "values": ["3610.00000000000"], "dims": ["sec"]}'

In [12]:
# 1[mm]*5[mm] + 1[inch^2] =
statements = '[{"type":"query","sympy":"implicit_param_0_0*implicit_param_0_1+implicit_param_0_2","units":"","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.001,"units_valid":true},{"name":"implicit_param_0_1","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.005,"units_valid":true},{"name":"implicit_param_0_2","dimensions":[0,2,0,0,0,0,0,0,0],"si_value":0.00064516,"units_valid":true}],"params":["implicit_param_0_0","implicit_param_0_1","implicit_param_0_2"]}]'
get_query_values(statements)

'{"error": null, "values": ["0.000650160000000000"], "dims": ["m^2"]}'

In [13]:
# incompatable dimensions
# 1[mm]*5[mm] + 1[inch] =
statements = '[{"type":"query","sympy":"implicit_param_0_0*implicit_param_0_1+implicit_param_0_2","units":"","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.001,"units_valid":true},{"name":"implicit_param_0_1","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.005,"units_valid":true},{"name":"implicit_param_0_2","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.0254,"units_valid":true}],"params":["implicit_param_0_0","implicit_param_0_1","implicit_param_0_2"]}]'
get_query_values(statements)


'{"error": null, "values": ["0.0254050000000000"], "dims": ["Dimension Error"]}'

In [14]:
# x = 1[m]
statements = '[{"type":"assignment","name":"x","sympy":"implicit_param_0_0","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":1,"units_valid":true}],"params":["implicit_param_0_0"]}]'
get_query_values(statements)

'{"error": null, "values": [""], "dims": [""]}'

In [15]:
# x = 1[m]
# y =
statements = '[{"type":"assignment","name":"x","sympy":"implicit_param_0_0","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":1,"units_valid":true}],"params":["implicit_param_0_0"]},{"type":"query","sympy":"y","units":"","implicitParams":[],"params":["y"]}]'
get_query_values(statements)

'{"error": null, "values": ["", ""], "dims": ["", ""]}'

In [16]:
# duplicate assignment
# x = 1[m]
# x = 2[m]
statements = '[{"type":"assignment","name":"x","sympy":"implicit_param_0_0","implicitParams":[{"name":"implicit_param_0_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":1,"units_valid":true}],"params":["implicit_param_0_0"]},{"type":"assignment","name":"x","sympy":"implicit_param_1_0","implicitParams":[{"name":"implicit_param_1_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":2,"units_valid":true}],"params":["implicit_param_1_0"]}]'
get_query_values(statements)

'{"error": "DuplicateAssignment", "values": null, "dims": null}'

In [17]:
# cyclical reference
# x = y
# y = x
statements = '[{"type":"assignment","name":"x","sympy":"y","implicitParams":[],"params":["y"]},{"type":"assignment","name":"y","sympy":"x","implicitParams":[],"params":["x"]}]'
get_query_values(statements)

Reference cycle detected


'{"error": "ReferenceCycle", "values": null, "dims": null}'

In [18]:
# 5 =
statements = '[{"type":"query","sympy":"5","units":"","implicitParams":[],"params":[]}]'
get_query_values(statements)


'{"error": null, "values": ["5.00000000000000"], "dims": [""]}'

In [19]:
# 5 =
# 5 + 10 =
statements = '[{"type":"query","sympy":"5","units":"","implicitParams":[],"params":[]},{"type":"query","sympy":"5+10","units":"","implicitParams":[],"params":[]}]'
get_query_values(statements)

'{"error": null, "values": ["5.00000000000000", "15.0000000000000"], "dims": ["", ""]}'

In [20]:
# undefined value
# x =
statements = '[{"type":"query","sympy":"x","units":"","implicitParams":[],"params":["x"]}]'
get_query_values(statements)

'{"error": null, "values": [""], "dims": [""]}'

In [21]:
# pi =
statements = '[{"type":"query","sympy":"pi","units":"","implicitParams":[],"params":[]}]'
get_query_values(statements)

'{"error": null, "values": ["3.14159265358979"], "dims": [""]}'

In [22]:
# cos(1) =
statements = '[{"type":"query","sympy":"cos(1)","units":"","implicitParams":[],"params":[]}]'
get_query_values(statements)

'{"error": null, "values": ["0.540302305868140"], "dims": ["Dimension Error"]}'

In [23]:
# quadratic equation from test suite that fails for dimension error even that dimensions are consistant
statements = '[{"type":"assignment","name":"x","sympy":"((-(b))+sqrt((b)**(2)-4*a*c))/(2*a)","implicitParams":[],"params":["b","b","a","c","a"]},{"type":"query","sympy":"x","units":"m","dimensions":[0,1,0,0,0,0,0,0,0],"units_valid":true,"implicitParams":[],"params":["x"]},{"type":"assignment","name":"a","sympy":"1","implicitParams":[],"params":[]},{"type":"assignment","name":"b","sympy":"(-(implicit_param_21_0))","implicitParams":[{"name":"implicit_param_21_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":5,"units_valid":true}],"params":["implicit_param_21_0"]},{"type":"assignment","name":"c","sympy":"implicit_param_22_0","implicitParams":[{"name":"implicit_param_22_0","dimensions":[0,2,0,0,0,0,0,0,0],"si_value":6,"units_valid":true}],"params":["implicit_param_22_0"]}]'
get_query_values(statements)

'{"error": null, "values": ["", "3.00000000000000", "", "", ""], "dims": ["", "m", "", "", ""]}'

In [24]:
# 5[mm]-4[mm] =
statements = '[{"type":"query","sympy":"implicit_param_6_0-implicit_param_6_1","units":"","implicitParams":[{"name":"implicit_param_6_0","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.005,"units_valid":true},{"name":"implicit_param_6_1","dimensions":[0,1,0,0,0,0,0,0,0],"si_value":0.004,"units_valid":true}],"params":["implicit_param_6_0","implicit_param_6_1"]}]'
get_query_values(statements)

'{"error": null, "values": ["0.00100000000000000"], "dims": ["m"]}'